In [ ]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, BasicAer
from qiskit.visualization import plot_bloch_multivector, plot_histogram

In [ ]:
def bitwise_complement(circuit,g,n):
    for i in range(0,n):
        circuit.x(g[i])

def subtraction(circuit,x,y,c,N):
    for i in range(N-1,-1,-1):
        circuit.cx(x[i],y[i])
    
    for i in range(N-2,0,-1):
        circuit.cx(x[i],x[i+1])
    
    for i in range(1,N):
        circuit.ccx(y[i-1],x[i-1],x[i])
        circuit.cx(x[i],y[i])
    
    circuit.ccx(y[N-1],x[N-1],c)
    
    for i in range(N-2,-1,-1):
        circuit.ccx(y[i],x[i],x[i+1])
    
    for i in range(1,N-1):
        circuit.cx(x[i],x[i+1])
    
    circuit.cx(x[N-1],c)
    
    for i in range(N-1,0,-1):
        circuit.cx(x[i],y[i])

In [ ]:
"""
    x is the register to increment with the control qubit as first element, so n qubits
    g is a register with n-1 dirty qubits
    g'-1 is the bitwise complement of g
    |x>|g> -> |x-g>|g> -> |x-g>|g'-1> -> |x-g-g'+1>|g'-1> -> |x+1>|g>
"""
def incremental_gate_controlled(circuit,x,g,n):
    if(n==2):
        circuit.cx(x[0],x[1])
    elif(n==3):
        circuit.ccx(x[0],x[1],x[2])
        circuit.cx(x[0],x[1])
    else:
        subtraction(circuit,g,x[:n-1],x[n-1],n-1)
        bitwise_complement(circuit,g,n-1)
        subtraction(circuit,g,x[:n-1],x[n-1],n-1)
        bitwise_complement(circuit,g,n-1)
        circuit.x(x[0])
        circuit.x(x[n-1])

In [ ]:
def incrementer(A,s):
    n=3
    x = QuantumRegister(n)
    c = QuantumRegister(1)
    g = QuantumRegister(n)
    clas_A = ClassicalRegister(n)
    clas_s = ClassicalRegister(n)
    clas_c = ClassicalRegister(1)
    circuit = QuantumCircuit(g,c,x,clas_A,clas_s,clas_c)
    s_A=bin(int(A))[2:].zfill(n)
    s_s=bin(int(s))[2:].zfill(n)
    for i in range(0, n):
        if s_A[i] == '1':
            circuit.x(x[n-1-i])
    for i in range(0, n):
        if s_s[i] == '1':
            circuit.x(g[n-1-i])
    circuit.x(c)
    incremental_gate_controlled(circuit,c[0:1]+x[0:n],g,n+1)
    for i in range(0,n):
        circuit.measure(x[i],clas_A[i])
    for i in range(0,n):
        circuit.measure(g[i],clas_s[i])
    circuit.measure(c,clas_c)
    backend = BasicAer.get_backend('qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_A = output[2]
    output_s = output[1]
    output_c = output[0]
    res_A = int(output_A, 2)
    res_s = int(output_s, 2)
    res_c = int(output_c, 2)
    print('A:',A,'s:',s)
    print('res_A:',res_A,'res_s:',res_s,'res_c:',res_c)
    if(res_c!=1 or res_A!=(A+1)%(2**n) or res_s!=s):
        print('found!')

In [ ]:
for i in range (0,2**3):
    for j in range (0,2**3):
        incrementer(i,j)
        print('-----------')